In [1]:
!pip install flask-ngrok


In [6]:
!pip install flask pyngrok openai pandas


In [10]:
from pyngrok import ngrok
ngrok.set_auth_token("35LFhPzJPW68glXwEf1f9lezzx9_7Fvgxr4u8q6jjJdPg1GpH")


In [16]:
!pip install flask pyngrok transformers torch pandas sqlalchemy


In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("retail.db")

# ------------------ CUSTOMERS ------------------
customers = pd.DataFrame({
    "customer_id": [1,2,3,4,5,6],
    "name": ["Alice","Bob","Charlie","Diana","Eve","Frank"],
    "region": ["North","South","East","West","North","South"],
    "age": [28, 35, 42, 30, 25, 38],
    "gender": ["F","M","M","F","F","M"]
})
customers.to_sql("customers", conn, index=False, if_exists="replace")

# ------------------ PRODUCTS ------------------
products = pd.DataFrame({
    "product_id": [101,102,103,104,105,106],
    "name": ["Smartphone","Laptop","Headphones","T-Shirt","Shoes","Blender"],
    "category": ["Electronics","Electronics","Electronics","Apparel","Apparel","Home"],
    "price": [700,1200,150,25,80,60]
})
products.to_sql("products", conn, index=False, if_exists="replace")

# ------------------ ORDERS ------------------
orders = pd.DataFrame({
    "order_id": [1001,1002,1003,1004,1005,1006,1007,1008,1009],
    "customer_id": [1,2,1,3,4,5,6,3,2],
    "product_id": [101,103,104,102,106,105,101,105,103],
    "quantity": [1,2,3,1,1,2,1,1,3],
    "order_date": [
        "2023-01-12","2023-02-08","2023-03-15",
        "2023-03-28","2023-04-01","2023-05-20",
        "2023-06-02","2023-06-18","2023-07-10"
    ],
    "region": ["North","South","North","East","West","North","South","East","South"]
})
orders.to_sql("orders", conn, index=False, if_exists="replace")

conn.close()
print("✅ retail.db created successfully with 3 tables!")


✅ retail.db created successfully with 3 tables!


In [3]:
import os
print(os.getcwd())  # shows current working directory
print(os.path.exists("retail.db"))  # should be True


/content
True


In [11]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import pandas as pd
import sqlite3
import os
import requests
import json

# ------------------ CONFIG ------------------
app = Flask(__name__)
previous_queries = []  # [(query, sql)]

# Groq LLM Config
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "GIVE YOUR KEY HERE")
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"
MODEL_NAME = "llama-3.1-8b-instant"

# ------------------ DATABASE SETUP ------------------
print("🧩 Initializing retail.db ...")
conn = sqlite3.connect("retail.db")

# CUSTOMERS
customers = pd.DataFrame({
    "customer_id": [1,2,3,4,5,6],
    "name": ["Alice","Bob","Charlie","Diana","Eve","Frank"],
    "region": ["North","South","East","West","North","South"],
    "age": [28, 35, 42, 30, 25, 38],
    "gender": ["F","M","M","F","F","M"]
})
customers.to_sql("customers", conn, index=False, if_exists="replace")

# PRODUCTS
products = pd.DataFrame({
    "product_id": [101,102,103,104,105,106],
    "name": ["Smartphone","Laptop","Headphones","T-Shirt","Shoes","Blender"],
    "category": ["Electronics","Electronics","Electronics","Apparel","Apparel","Home"],
    "price": [700,1200,150,25,80,60]
})
products.to_sql("products", conn, index=False, if_exists="replace")

# ORDERS
orders = pd.DataFrame({
    "order_id": [1001,1002,1003,1004,1005,1006,1007,1008,1009],
    "customer_id": [1,2,1,3,4,5,6,3,2],
    "product_id": [101,103,104,102,106,105,101,105,103],
    "quantity": [1,2,3,1,1,2,1,1,3],
    "order_date": [
        "2023-01-12","2023-02-08","2023-03-15",
        "2023-03-28","2023-04-01","2023-05-20",
        "2023-06-02","2023-06-18","2023-07-10"
    ],
    "region": ["North","South","North","East","West","North","South","East","South"]
})
orders.to_sql("orders", conn, index=False, if_exists="replace")
conn.close()
print("✅ retail.db created successfully with 3 tables!")

# ------------------ Schema fetching function ------------------
def get_schema_info(db_path="retail.db"):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    schema_info = {}
    tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

    for t in tables:
        table_name = t[0]
        if table_name == "sqlite_sequence":  # skip internal tables
            continue
        columns = cursor.execute(f"PRAGMA table_info({table_name})").fetchall()
        schema_info[table_name] = [(col[1], col[2]) for col in columns]

    conn.close()
    return schema_info

# ------------------ LLM PROMPT ------------------
BASE_PROMPT = """
You are an expert SQL generator for a SQLite retail analytics database.

Database schema:
- customers(customer_id INTEGER, name TEXT, region TEXT, age INTEGER, gender TEXT)
- products(product_id INTEGER, name TEXT, category TEXT, price REAL)
- orders(order_id INTEGER, customer_id INTEGER, product_id INTEGER, quantity INTEGER, order_date TEXT, region TEXT)

Relationships:
- orders.customer_id → customers.customer_id
- orders.product_id → products.product_id

Rules:
1. Use valid SQLite syntax.
2. Use JOINs when multiple tables are involved.
3. Compute total sales or revenue as (quantity * price).
4. Use GROUP BY for aggregations.
5. Use WHERE for filters (e.g., category, region, year).
6. NEVER invent new tables or columns.
7. Output ONLY SQL — no explanations.

User: {user_query}
SQL:
""".strip()

# ------------------ LLM CALL ------------------
def generate_sql_from_groq(user_query: str) -> str:
    prompt = BASE_PROMPT.format(user_query=user_query)
    headers = {"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"}
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You convert natural language to SQL."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
    }

    resp = requests.post(GROQ_URL, headers=headers, data=json.dumps(payload))
    if resp.status_code != 200:
        print("Groq API error:", resp.text)
        return "-- ERROR calling LLM"

    sql_query = resp.json()["choices"][0]["message"]["content"].strip()
    if sql_query.startswith("```"):
        sql_query = sql_query.strip("`").replace("sql", "", 1).strip()

    print("Generated SQL:", sql_query)
    return sql_query

# ------------------ RUN QUERY ------------------
def run_query(sql_query: str) -> str:
    try:
        conn = sqlite3.connect("retail.db")
        clean_sql = " ".join(sql_query.split())
        df = pd.read_sql(clean_sql, conn)
        conn.close()
        if df.empty:
            return "<p>No results found.</p>"
        return df.to_html(index=False, classes="data-table")
    except Exception as e:
        return f"<p style='color:red;'>Error executing query: {e}</p>"

# ------------------ HTML TEMPLATE ------------------
TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Prompt2SQL</title>
<style>
body { font-family: 'Inter', sans-serif; background: #f6f8fa; margin: 0; color: #222; }
header { background: linear-gradient(90deg, #7b2ff7, #f107a3); color: white; text-align: center; padding: 1.5rem; font-size: 1.8rem; font-weight: 700; box-shadow: 0 2px 10px rgba(0,0,0,0.15); }
main { max-width: 900px; margin: 40px auto; background: white; border-radius: 16px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); padding: 2rem; }
textarea { width: 100%; height: 80px; font-size: 1rem; padding: 10px; border-radius: 8px; border: 1px solid #ccc; margin-top: 10px; }
button { margin-top: 15px; background: linear-gradient(90deg, #7b2ff7, #f107a3); color: white; border: none; padding: 10px 18px; font-size: 1rem; border-radius: 8px; cursor: pointer; }
button:hover { transform: scale(1.03); }
details { margin-bottom: 10px; background: #fff; border-radius: 8px; border: 1px solid #ddd; padding: 8px 12px; }
summary { cursor: pointer; font-weight: 600; }
pre { background: #1e1e1e; color: #f8f8f2; padding: 15px; border-radius: 8px; overflow-x: auto; font-size: 0.9rem; }
table.data-table { border-collapse: collapse; width: 100%; margin-top: 15px; border-radius: 8px; overflow: hidden; }
th, td { border: 1px solid #eee; padding: 10px; text-align: left; }
th { background-color: #f7f7f7; }
footer { text-align: center; margin-top: 40px; font-size: 0.9rem; color: #666; }
</style>
</head>
<body>
<header>Prompt2SQL</header>
<main>
<h2>Ask your question...</h2>
<form method="POST" action="/generate">
<textarea name="query" placeholder="e.g. Show total sales by region for 2023"></textarea><br>
<button type="submit">Generate SQL</button>
</form>

{% if schema %}
<div class="output-section" style="margin-top:30px;">
    <h3>Database Schema Overview</h3>
    {% for table, cols in schema.items() %}
    <details>
        <summary>{{ table }}</summary>
        <ul>
        {% for col, dtype in cols %}
            <li>{{ col }} ({{ dtype }})</li>
        {% endfor %}
        </ul>
    </details>
    {% endfor %}
</div>
{% endif %}

{% if sql %}
<div class="output-section">
    <h3>Generated SQL for the query:</h3>
    <p style="font-style: italic; color: #555;">“{{ user_query }}”</p>
    <pre>{{ sql }}</pre>
</div>
{% endif %}

{% if result %}
<div class="output-section">
    <h3>Query Result</h3>
    {{ result | safe }}
</div>
{% endif %}

{% if previous %}
<div class="output-section">
    <h3>Previous Queries</h3>
    {% for q, s in previous %}
    <details><summary>{{ q }}</summary><pre>{{ s }}</pre></details>
    {% endfor %}
</div>
{% endif %}
</main>
<footer><p>🚀 Built with <strong>Groq LLM (LLaMA 3.1)</strong> | Flask + SQLite Demo</p></footer>
</body>
</html>
"""

# ------------------ ROUTES ------------------
@app.route("/", methods=["GET"])
def index():
    schema = get_schema_info()
    return render_template_string(TEMPLATE, schema=schema, previous=previous_queries)

@app.route("/generate", methods=["POST"])
def generate():
    user_q = request.form["query"].strip()
    sql = generate_sql_from_groq(user_q)
    result = run_query(sql)
    if user_q:
        previous_queries.insert(0, (user_q, sql))
        del previous_queries[5:]
    schema = get_schema_info()
    return render_template_string(TEMPLATE, sql=sql, result=result, user_query=user_q, previous=previous_queries, schema=schema)

# ------------------ START ------------------
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url.public_url)
app.run(port=5000)


🧩 Initializing retail.db ...
✅ retail.db created successfully with 3 tables!
🚀 Public URL: https://santos-oarlike-ronnie.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 20:59:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2025 20:59:55] "POST /generate HTTP/1.1" 200 -


Generated SQL: SELECT p.product_id, p.name, SUM(o.quantity * p.price) AS total_sales
FROM orders o
JOIN products p ON o.product_id = p.product_id
WHERE STRFTIME('%Y', o.order_date) = '2023'
GROUP BY p.product_id, p.name
ORDER BY total_sales DESC
LIMIT 1;


SANITY CHECKS

In [4]:
import sqlite3, pandas as pd
conn = sqlite3.connect("retail.db")
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))
conn.close()


        name
0  customers
1   products
2     orders


In [6]:
conn = sqlite3.connect("retail.db")
print(pd.read_sql("SELECT p.name, p.category FROM products p;", conn))
conn.close()


         name     category
0  Smartphone  Electronics
1      Laptop  Electronics
2  Headphones  Electronics
3     T-Shirt      Apparel
4       Shoes      Apparel
5     Blender         Home
